In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql.functions import (
    col, collect_set, concat, count, current_timestamp, explode,
    lit, lower, size, split, udf
)
from pyspark.sql.types import (
    ArrayType, BooleanType, StringType,
    StructField, StructType, IntegerType
)
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [3]:
# Get spark context
sc = SparkContext.getOrCreate(conf=conf) # conf is provided by sparkmonitor
spark = SparkSession(sc)

## Orange Book Data (NDA, Patents)

This will work with the `./FDA_drug_patents.csv` file in this repo.

In [4]:
orange_book_df = spark.read.csv('./FDA_drug_patents.csv', header=True, inferSchema=True)
# Explore Columns
orange_book_df.printSchema()
orange_book_df.show()

root
 |-- _c0: integer (nullable = true)
 |-- edition: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- application_number: integer (nullable = true)
 |-- product_number: integer (nullable = true)
 |-- patent_number: string (nullable = true)
 |-- patent_expiration: timestamp (nullable = true)
 |-- use_code: string (nullable = true)
 |-- active_ingredient: string (nullable = true)
 |-- trade_name: string (nullable = true)
 |-- DS: string (nullable = true)
 |-- DP: string (nullable = true)
 |-- ped_extension: double (nullable = true)
 |-- delist_requested: string (nullable = true)

+---+-------+----------------+------------------+--------------+-------------+-------------------+--------+-----------------+----------+---+---+-------------+----------------+
|_c0|edition|application_type|application_number|product_number|patent_number|  patent_expiration|use_code|active_ingredient|trade_name| DS| DP|ped_extension|delist_requested|
+---+-------+----------------+-

In [5]:
sample_df = (orange_book_df
                .filter(orange_book_df['active_ingredient'].isin('MORPHINE SULFATE', 'MINOXIDIL'))
                .select('patent_number', 'application_number'))

In [6]:
appln_numbers = sample_df.select('application_number').distinct()
appln_numbers.show()

+------------------+
|application_number|
+------------------+
|            204223|
|             21260|
|             21671|
|             20616|
|             21812|
|             19501|
+------------------+



In [7]:
appln_numbers_list = list(map(lambda x: x.application_number, appln_numbers.collect()))
appln_numbers_list

[204223, 21260, 21671, 20616, 21812, 19501]

In [8]:
patent_numbers = sample_df.select('patent_number').distinct()
patent_numbers.show()

+-------------+
|patent_number|
+-------------+
|      5931089|
|      5202128|
|      5997899|
|      5962016|
|      6071534|
|      6066339|
|      5931809|
|      6946120|
|      5891467|
|      9072781|
|      6171613|
|      9192608|
|      4139619|
|      5807572|
|      6241999|
|      4596812|
|      6193998|
|      5378474|
|      5723147|
+-------------+



In [9]:
patent_numbers_list = list(map(lambda x: x.patent_number, patent_numbers.collect()))
patent_numbers_list

['5931089',
 '5202128',
 '5997899',
 '5962016',
 '6071534',
 '6066339',
 '5931809',
 '6946120',
 '5891467',
 '9072781',
 '6171613',
 '9192608',
 '4139619',
 '5807572',
 '6241999',
 '4596812',
 '6193998',
 '5378474',
 '5723147']

## OpenFDA Data (NDA, SPL Set ID)

This will work with the `./spl_id_label_nda.csv` file in this repo.

In [10]:
spls = spark.read.csv('./spl_id_label_nda.csv', header=True, inferSchema=True)
spls.printSchema()

root
 |-- effective_time: integer (nullable = true)
 |-- spl_id: string (nullable = true)
 |-- spl_set_id: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- manufacturer_name: string (nullable = true)
 |-- nda: string (nullable = true)



In [11]:
# Format the NDA application numbers to 'NDAxxxxxx'
appln_numbers_string_list = list(map(lambda x: 'NDA' + str(x).zfill(6), appln_numbers_list))

In [12]:
set_ids = (spls
           .where(col('nda').isin(appln_numbers_string_list))
           .select('spl_set_id')
           .distinct())
set_ids.show(truncate=False)

+------------------------------------+
|spl_set_id                          |
+------------------------------------+
|b62ee1ef-db55-417a-a02e-b103890f9c4b|
|8bf0000c-95f3-4a4d-830b-f5ac1539823d|
|4d328537-b7f5-43cc-9837-c5a0c6c390f8|
|582f42e5-444e-4246-af8c-e7e28097c69a|
|b5cee013-000f-4e35-a284-1f58add31b4d|
|1b5e2860-6855-4a65-8bbc-e064172a1adf|
|1f53de80-efc8-4930-b3e3-fba0d026af05|
|6fe62c06-5bab-4fc8-b546-ff015a1fc40c|
+------------------------------------+



In [13]:
set_ids_list = list(map(lambda x: x.spl_set_id, set_ids.collect()))
set_ids_list

['b62ee1ef-db55-417a-a02e-b103890f9c4b',
 '8bf0000c-95f3-4a4d-830b-f5ac1539823d',
 '4d328537-b7f5-43cc-9837-c5a0c6c390f8',
 '582f42e5-444e-4246-af8c-e7e28097c69a',
 'b5cee013-000f-4e35-a284-1f58add31b4d',
 '1b5e2860-6855-4a65-8bbc-e064172a1adf',
 '1f53de80-efc8-4930-b3e3-fba0d026af05',
 '6fe62c06-5bab-4fc8-b546-ff015a1fc40c']

## Patent Data Import

For this, obtain and unzip the USPTO Patent Claim Research dataset from: https://developer.uspto.gov/product/patent-and-patent-application-claims-data-stata-dta-and-ms-excel-csv

Place the unzipped .csv in a folder, accessible to the notebook. The filepath in the notebook is hardcoded to: `data/PatentClaims/patent_claims_fulltext.csv`.

**NOTE**: The unzipped data is about 30GB in size.

In [15]:
patents = spark.read.csv('data/PatentClaims/patent_claims_fulltext.csv', header=True, inferSchema=True)
patents.printSchema()

root
 |-- pat_no: string (nullable = true)
 |-- claim_no: integer (nullable = true)
 |-- claim_txt: string (nullable = true)
 |-- dependencies: string (nullable = true)
 |-- ind_flg: integer (nullable = true)
 |-- appl_id: integer (nullable = true)



In [16]:
selected_patents = patents.where(col('pat_no').isin(patent_numbers_list))
# Cache this to avoid recompute!
selected_patents.cache()

DataFrame[pat_no: string, claim_no: int, claim_txt: string, dependencies: string, ind_flg: int, appl_id: int]

In [17]:
# count all rows (one row per claim)
selected_patents.count()

349

In [18]:
selected_patents.select('pat_no').distinct().count()

17

We have 17 of the 19 patents matched using the Orange Book - the 2 newer ones (9XXXXXX) are not in the research dataset.

In [20]:
selected_patents.show(10)

+-------+--------+--------------------+------------+-------+-------+
| pat_no|claim_no|           claim_txt|dependencies|ind_flg|appl_id|
+-------+--------+--------------------+------------+-------+-------+
|4139619|       7|7. A process for ...|        null|      1|5826180|
|4139619|       1|1. A topical comp...|        null|      1|5826180|
|4139619|       4|4. A process for ...|        null|      1|5826180|
|4139619|       8|8. The process of...|           7|      0|5826180|
|4139619|       9|9. The process of...|           7|      0|5826180|
|4139619|       5|5. The process of...|           4|      0|5826180|
|4139619|       3|3. The compositio...|           1|      0|5826180|
|4139619|       6|6. The process of...|           4|      0|5826180|
|4139619|       2|2. The compositio...|           1|      0|5826180|
|4596812|      13|13. In a pharmace...|        null|      1|6181959|
+-------+--------+--------------------+------------+-------+-------+
only showing top 10 rows



There is no date info. So, using patent expiration from the Orange Book data instead.

In [24]:
orange_book_dates = orange_book_df.select('patent_number', 'patent_expiration').distinct()
joined_df = (selected_patents
             .join(orange_book_dates, orange_book_dates.patent_number==selected_patents.pat_no)
             .orderBy('pat_no', 'claim_no'))
joined_df.show(3)

+-------+--------+--------------------+------------+-------+-------+-------------+-------------------+
| pat_no|claim_no|           claim_txt|dependencies|ind_flg|appl_id|patent_number|  patent_expiration|
+-------+--------+--------------------+------------+-------+-------+-------------+-------------------+
|4139619|       1|1. A topical comp...|        null|      1|5826180|      4139619|1996-02-13 00:00:00|
|4139619|       2|2. The compositio...|           1|      0|5826180|      4139619|1996-02-13 00:00:00|
|4139619|       3|3. The compositio...|           1|      0|5826180|      4139619|1996-02-13 00:00:00|
+-------+--------+--------------------+------------+-------+-------+-------------+-------------------+
only showing top 3 rows



In [25]:
# Convert to in-memory object
patent_claim_list = joined_df.collect()

In [46]:
# Clear the cached data
selected_patents.unpersist()

DataFrame[pat_no: string, claim_no: int, claim_txt: string, dependencies: string, ind_flg: int, appl_id: int]

### Import Patents to MongoDB

If both MongoDB and Jupyter Notebooks are being run from Docker container, start one service on the network of the other. For example, the below config can be added to the `docker-compose` set up for MongoDB:

```
networks:
  default:
    external:
      name: ds_notebook_default
```

In [32]:
from collections import defaultdict
patent_id_dict = defaultdict(list)
patent_expiration_dict = {}

# Claims are already ordered in the patent_claim_list
for row in patent_claim_list:
    patent_id_dict[row.patent_number].append({
        'claim_number': row.claim_no,
        'claim_text': row.claim_txt,
        'dependencies': row.dependencies
    })
    patent_expiration_dict[row.patent_number] = str(row.patent_expiration)[:10]

In [45]:
# Connect to MongoDB and save data
import pymongo
mongo = pymongo.MongoClient('mongodb://mongo_local:27017')
db = mongo['mockdata']
coll = db['patents']

for patent_id in patent_id_dict:
    # One record per patent
    record = {
        'patent_number': patent_id,
        'expiration_date': patent_expiration_dict[patent_id],
        'claims': patent_id_dict[patent_id]
    }
    # Upsert into Mongo
    coll.update({'patent_number': patent_id}, record, upsert=True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  from ipykernel import kernelapp as app
